# Demo of the Free Throw Shot Predictor

This notebook runs through the entire sequence of importing, processing, and predicting on a clip named "demo.mp4". 

---

Necessary Imports and Helper Functions

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import skvideo.io
import argparse
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches
import imageio
import pandas as pd
from moviepy.editor import *
from IPython.display import HTML, display
import time
import os

In [18]:
def downsample_video(video: str, num_frames: int):
    """
    Downsamples a given video clip to a given number of frames.
    Returns an generator object to each frame in the video
    Parameters
    ----------
    video:
        - filename of video without the .mp4 extension
    num_frames:
        - the desired number of frames to extract
    output_file:
        - the filename to put the downsampled video in, with the mp4 extension omitted.
    """
    clip = VideoFileClip(video)
    duration = clip.duration
    clip_time = 1.5
    video_clipped = clip.subclip(0, clip_time)
    video_downsampled = video_clipped.set_fps(num_frames/clip_time)
    return video_downsampled.iter_frames() 

In [19]:
def load_model(url):
    """
    Load model from tensorflow hub

    Parameters
    ----------
    url: str
        - url of the model

    Returns
    -------
    The desired model
    """
    module = hub.load(url)
    return module

In [20]:
def movenet(module, input_image):
    """
    Runs pose detection on an input image

    Parameters
    ----------
    module:
        - model loaded in from tensorflow_hub
    input_image: tf.tensor
        - input image of shape (1, h, w, 3) that represents input image pixels. Height and width of the image should already be resized to match the expected input resolution of the model before passing into this function

    Returns
    -------
    keypoints_with_scores: np.ndarray
        - numpy array of size (1, 1, 17, 3) representing keypoint coordinates and their confidence scores
    """
    model = module.signatures['serving_default']

    input_image = tf.cast(input_image, dtype=tf.int32)

    outputs = model(input_image)

    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

Import Clip and run Movenet model to extract a pose vector

In [21]:
videogen = downsample_video('demo.mp4', 60)
model = load_model("./saved_model")
input_size = 256
row = np.asarray([])
for frame in videogen:
    image = frame
    input_image = tf.expand_dims(image, axis = 0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
    keypoints_and_scores = movenet(model, input_image)
    keypoints = [x for i,x in enumerate(keypoints_and_scores.flatten()) if i%3!=2 or i==0]
    row = np.append(row, keypoints)

Process the pose feature vector into the correct format

In [22]:
clip = row
clip = np.reshape(clip,(1,-1)) #feature vector for the clip
numrows = int((clip.shape[0]*clip.shape[1])/34) #should be 60 for our standard, but this keeps it dynamic
numcols = int((clip.shape[0]*clip.shape[1])/numrows) #should always end up as 34
clip = np.reshape(clip,(numrows, numcols))
print(clip.shape)

(60, 34)


Import our pre-trained and saved model, and run prediction

In [23]:
import pickle
with open('r2cboost.pk1','rb') as file:
    cbm = pickle.load(file)

#classify each row (frame) in clip
clipPreds = cbm.predict(clip)
clipSoftPreds = cbm.predict_proba(clip)[:,1]

#create singular prediction values, both hard prediction and soft score
prediction = np.sum(clipPreds)
if(prediction < (clipPreds.shape[0]/2)):
    prediction = 0
else:
    prediction = 1

softprediction = np.average(clipSoftPreds)

Output prediction

In [24]:
print("PREDICTION (0=Miss, 1=Make): "+str(prediction))
print("SOFT SCORE (Probability of being a make): "+str(softprediction))

PREDICTION (0=Miss, 1=Make): 0
SOFT SCORE (Probability of being a make): 0.27218008811313155
